In [1]:
import pandas as pd

In [2]:
df_final=pd.read_csv('df_final.csv')

In [4]:
df_final['Genero']

0              italian hip hop, italian pop, trap italiana
1                                                      NaN
2        ambient, braindance, electronica, intelligent ...
3                           jam band, neo mellow, pop rock
4                        argentine hip hop, trap argentino
                               ...                        
16861                                        j-pop, j-rock
16862                                                k-pop
16863                                    k-pop, korean r&b
16864                              k-pop, k-pop girl group
16865                                    k-pop, korean r&b
Name: Genero, Length: 16866, dtype: object

In [5]:
df_final['Stream'].describe()

count    1.686600e+04
mean     1.362465e+08
std      2.433885e+08
min      6.574000e+03
25%      1.748466e+07
50%      4.993142e+07
75%      1.391041e+08
max      3.386520e+09
Name: Stream, dtype: float64

In [6]:
# Agrupar por 'Genero' y contar el número de tracks
df_cluster_count = df_final.groupby('Cluster', as_index=False).size()

# Renombrar las columnas para mayor claridad
df_cluster_count.columns = ['Cluster', 'Num_Tracks']

# Ordenar por número de tracks en orden descendente
df_cluster_count = df_cluster_count.sort_values(by='Num_Tracks', ascending=False)

# Reiniciar el índice para un formato limpio
df_cluster_count.reset_index(drop=True, inplace=True)


In [7]:
df_cluster_count

,Cluster,Num_Tracks
0,1,12943
1,3,944
2,5,791
3,7,606
4,4,519
5,6,393
6,2,384
7,0,286


El cluster 1 incluye todos los generos que no están dentro de los principales, por lo tanto no entrará en el análsis

## Cálculo de quartiles del Stream

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

# Lista de clusters a evaluar
clusters = [0, 2, 3, 4, 5, 6,7]

# Diccionario para almacenar resultados
test = {}

for cluster in clusters:
    # Filtrar por cluster y hacer una copia para evitar errores
    df_cluster = df_final[df_final['Cluster'] == cluster].copy()

    # Calcular el tercer cuartil (Q3)
    q3 = df_cluster['Stream'].quantile(0.75)

    # Definir las categorías 0 para las canciones con menos de q3 de stream y 1 para las de mayor de q3, estás serían las más exitosas
    bins = [0, q3, df_cluster['Stream'].max()]
    labels = [0, 1]

    df_cluster['Categoria'] = pd.cut(df_cluster['Stream'], bins=bins, labels=labels, include_lowest=True)

    # Verificar si hay suficientes muestras en la categoría 1
    count_cat1 = df_cluster['Categoria'].value_counts().get(1, 0)

    if count_cat1 == 0:
        print(f"⚠️ cluster {cluster}: No hay suficientes muestras en la categoría 1, saltando...")
        continue

    # Filtrar por categorías
    categoria_0 = df_cluster[df_cluster['Categoria'] == 0]
    categoria_1 = df_cluster[df_cluster['Categoria'] == 1]

    # Muestrear la categoría 0 con el mismo número de elementos que la categoría 1
    categoria_0_sample = categoria_0.sample(n=min(count_cat1, len(categoria_0)), random_state=42)

    # Concatenar para obtener un dataset balanceado
    df_balanceado = pd.concat([categoria_0_sample, categoria_1]).sample(frac=1, random_state=42).reset_index(drop=True)

    # Definir variables predictoras y objetivo
    X = df_balanceado[['Acousticness', 'Danceability', 'Duration_min', 'Energy',
                       'Instrumentalness', 'Key', 'Liveness', 'Loudness',
                       'Speechiness', 'Tempo', 'Valence']]
    y = df_balanceado['Categoria']

    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Evaluación con LazyClassifier
    lazy_clf = LazyClassifier(verbose=0, ignore_warnings=False, custom_metric=None)
    models, predictions = lazy_clf.fit(X_train, X_test, y_train, y_test)

    # Guardar resultados en el diccionario
    test[cluster] = models

print("✅ Evaluación completada para todos los clusters.")

  6%|▋         | 2/32 [00:00<00:01, 16.54it/s]

CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X).


 41%|████      | 13/32 [00:00<00:00, 26.58it/s]

FixedThresholdClassifier model failed to execute
FixedThresholdClassifier.__init__() missing 1 required positional argument: 'estimator'


 72%|███████▏  | 23/32 [00:00<00:00, 28.74it/s]

...

SelfTrainingClassifier model failed to execute
You must pass an estimator to SelfTrainingClassifier. Use `estimator`.
StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
TunedThresholdClassifierCV model failed to execute
TunedThresholdClassifierCV.__init__() missing 1 required positional argument: 'estimator'
XGBClassifier model failed to execute
'super' object has no attribute '__sklearn_tags__'
[LightGBM] [Info] Number of positive: 77, number of negative: 79
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000037 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 517
[LightGBM] [Info] Number of data points in the train set: 156, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493590 -> initscore=-0.025642
[LightGBM] [Info] Start training from score -0.025642
[LightGBM] [Warning] No further splits wi

 28%|██▊       | 9/32 [00:00<00:00, 32.23it/s]

CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X).
FixedThresholdClassifier model failed to execute
FixedThresholdClassifier.__init__() missing 1 required positional argument: 'estimator'


100%|██████████| 32/32 [00:00<00:00, 39.36it/s]

SelfTrainingClassifier model failed to execute
You must pass an estimator to SelfTrainingClassifier. Use `estimator`.
StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
TunedThresholdClassifierCV model failed to execute
TunedThresholdClassifierCV.__init__() missing 1 required positional argument: 'estimator'
XGBClassifier model failed to execute
'super' object has no attribute '__sklearn_tags__'
[LightGBM] [Info] Number of positive: 118, number of negative: 125
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000048 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 756
[LightGBM] [Info] Number of data points in the train set: 243, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.485597 -> initscore=-0.057629
[LightGBM] [Info] Start training from score -0.057629
[LightGBM] [Warning] No further splits 

In [26]:
for i in test.keys():
    print(f'Modelos del cluster {i}')
    display(test[i])  # Esto imprimirá los resultados de LazyClassifier

Modelos del cluster 0


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.69,0.70,0.70,0.69,0.19
BaggingClassifier,0.66,0.66,0.66,0.66,0.03
RidgeClassifier,0.62,0.65,0.65,0.61,0.02
LogisticRegression,0.62,0.65,0.65,0.61,0.03
LinearDiscriminantAnalysis,0.62,0.65,0.65,0.61,0.02
RidgeClassifierCV,0.62,0.65,0.65,0.61,0.02
LinearSVC,0.62,0.65,0.65,0.61,0.02
DecisionTreeClassifier,0.62,0.64,0.64,0.62,0.01
CalibratedClassifierCV,0.55,0.62,0.62,0.49,0.04


Modelos del cluster 2


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.69,0.69,0.69,0.69,0.12
ExtraTreesClassifier,0.67,0.68,0.68,0.66,0.08
LabelPropagation,0.64,0.67,0.67,0.63,0.01
LabelSpreading,0.64,0.67,0.67,0.63,0.01
AdaBoostClassifier,0.64,0.64,0.64,0.64,0.08
CalibratedClassifierCV,0.62,0.63,0.63,0.61,0.03
LinearDiscriminantAnalysis,0.62,0.63,0.63,0.61,0.01
RidgeClassifier,0.62,0.63,0.63,0.61,0.01
RidgeClassifierCV,0.62,0.63,0.63,0.61,0.01


Modelos del cluster 3


...

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.70,0.72,0.72,0.70,0.01
LinearSVC,0.70,0.72,0.72,0.70,0.02
RidgeClassifier,0.70,0.72,0.72,0.70,0.01
LogisticRegression,0.69,0.71,0.71,0.68,0.02
RidgeClassifierCV,0.68,0.69,0.69,0.67,0.02
CalibratedClassifierCV,0.62,0.66,0.66,0.60,0.03
QuadraticDiscriminantAnalysis,0.64,0.65,0.65,0.64,0.01
SVC,0.61,0.63,0.63,0.61,0.01
PassiveAggressiveClassifier,0.56,0.60,0.60,0.51,0.01


Modelos del cluster 6


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
DecisionTreeClassifier,0.55,0.55,0.55,0.55,0.01
LGBMClassifier,0.53,0.53,0.53,0.52,0.02
LabelPropagation,0.53,0.53,0.53,0.53,0.01
LabelSpreading,0.53,0.53,0.53,0.53,0.01
KNeighborsClassifier,0.53,0.52,0.52,0.51,0.02
BaggingClassifier,0.50,0.50,0.50,0.50,0.03
NearestCentroid,0.50,0.50,0.50,0.50,0.01
DummyClassifier,0.47,0.50,0.50,0.31,0.01
NuSVC,0.50,0.50,0.50,0.50,0.01


Modelos del cluster 7


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,0.66,0.65,0.65,0.65,0.01
ExtraTreesClassifier,0.62,0.62,0.62,0.62,0.09
ExtraTreeClassifier,0.61,0.61,0.61,0.61,0.02
SGDClassifier,0.61,0.60,0.60,0.60,0.01
DecisionTreeClassifier,0.59,0.59,0.59,0.59,0.01
NuSVC,0.59,0.59,0.59,0.59,0.02
RandomForestClassifier,0.59,0.59,0.59,0.59,0.13
QuadraticDiscriminantAnalysis,0.61,0.58,0.58,0.58,0.01
LGBMClassifier,0.59,0.58,0.58,0.58,0.03


## Opirmización de los modelos
A continuación veremos la optimización de los modelos de algunos clusters

## Cluster 3

In [32]:

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import make_classification

cluster=3

# Generar datos sintéticos (cambia esto por tu dataset real)
df_cluster = df_final[df_final['Cluster'] == cluster].copy()

# Calcular el tercer cuartil (Q3)
q3 = df_cluster['Stream'].quantile(0.75)

# Definir las categorías 0 para las canciones con menos de q3 de stream y 1 para las de mayor de q3, estás serían las más exitosas
bins = [0, q3, df_cluster['Stream'].max()]
labels = [0, 1]

df_cluster['Categoria'] = pd.cut(df_cluster['Stream'], bins=bins, labels=labels, include_lowest=True)

# Ver la distribución de las categorías
print(df_cluster['Categoria'].value_counts())




Categoria
0    708
1    236
Name: count, dtype: int64


In [33]:
# Verificar si hay suficientes muestras en la categoría 1
count_cat1 = df_cluster['Categoria'].value_counts().get(1, 0)



# Filtrar por categorías
categoria_0 = df_cluster[df_cluster['Categoria'] == 0]
categoria_1 = df_cluster[df_cluster['Categoria'] == 1]

# Muestrear la categoría 0 con el mismo número de elementos que la categoría 1
categoria_0_sample = categoria_0.sample(n=min(count_cat1, len(categoria_0)), random_state=42)

# Concatenar para obtener un dataset balanceado
df_balanceado = pd.concat([categoria_0_sample, categoria_1]).sample(frac=1, random_state=42).reset_index(drop=True)
print(df_balanceado['Categoria'].value_counts())

Categoria
0    236
1    236
Name: count, dtype: int64


In [34]:
# Definir variables predictoras y objetivo
X = df_balanceado[['Acousticness', 'Danceability', 'Duration_min', 'Energy',
                       'Instrumentalness', 'Key', 'Liveness', 'Loudness',
                       'Speechiness', 'Tempo', 'Valence']]
y = df_balanceado['Categoria']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [14]:
# Importar librerías
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# 1. Dividir datos en entrenamiento y prueba (asumiendo que tienes X e y definidos)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 2. Crear pipeline con escalado y modelo
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Escalado de características
    ('svc', SVC(random_state=42))  # Modelo SVC
])

# 3. Definir parámetros para Grid Search
param_grid = {
    'svc__C': [0.1, 1, 10, 100],  # Parámetro de regularización
    'svc__kernel': ['linear', 'rbf', 'poly'],
    'svc__gamma': ['scale', 'auto', 0.01, 0.1, 1],  # Ancho del kernel
    'svc__class_weight': [None, 'balanced']  # Manejo de clases desbalanceadas
}

# 4. Configurar Grid Search con validación cruzada estratificada
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='balanced_accuracy',  # Métrica clave para optimizar
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),  # 5 folds estratificados
    n_jobs=-1,  # Usar todos los núcleos del procesador
    verbose=1  # Mostrar progreso
)

# 5. Ejecutar Grid Search en datos de entrenamiento
grid_search.fit(X_train, y_train)

# 6. Evaluar resultados
print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)

print("\nMétricas en entrenamiento (CV):")
print(f"Mejor Balanced Accuracy: {grid_search.best_score_:.4f}")

# 7. Evaluar en conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nReporte de clasificación en Test:")
print(classification_report(y_test, y_pred))

# 8. (Opcional) Guardar modelo entrenado
# import joblib
# joblib.dump(best_model, 'mejor_modelo_svc.pkl')

Fitting 5 folds for each of 120 candidates, totalling 600 fits



Mejores parámetros encontrados:
{'svc__C': 1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}

Métricas en entrenamiento (CV):
Mejor Balanced Accuracy: 0.5861

Reporte de clasificación en Test:
              precision    recall  f1-score   support

           0       0.67      0.75      0.71        48
           1       0.71      0.62      0.66        47

    accuracy                           0.68        95
   macro avg       0.69      0.68      0.68        95
weighted avg       0.69      0.68      0.68        95



In [21]:
# Importar librerías
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# 1. Dividir datos en entrenamiento y prueba (asumiendo que tienes X e y definidos)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 2. Crear pipeline con escalado y modelo
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Escalado de características
    ('svc', SVC(random_state=42))  # Modelo SVC
])

# 3. Definir parámetros para Grid Search
param_grid = {
    'svc__C': [0.5, 1, 2, 3],  # Parámetro de regularización
    'svc__kernel': ['rbf'],
    'svc__gamma': ['scale', 'auto', 0.01, 0.02, 0.03],  # Ancho del kernel
    'svc__class_weight': [None, 'balanced']  # Manejo de clases desbalanceadas
}

# 4. Configurar Grid Search con validación cruzada estratificada
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='balanced_accuracy',  # Métrica clave para optimizar
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),  # 5 folds estratificados
    n_jobs=-1,  # Usar todos los núcleos del procesador
    verbose=1  # Mostrar progreso
)

# 5. Ejecutar Grid Search en datos de entrenamiento
grid_search.fit(X_train, y_train)

# 6. Evaluar resultados
print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)

print("\nMétricas en entrenamiento (CV):")
print(f"Mejor Balanced Accuracy: {grid_search.best_score_:.4f}")

# 7. Evaluar en conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nReporte de clasificación en Test:")
print(classification_report(y_test, y_pred))

# 8. (Opcional) Guardar modelo entrenado
# import joblib
# joblib.dump(best_model, 'mejor_modelo_svc.pkl')

Fitting 5 folds for each of 40 candidates, totalling 200 fits



Mejores parámetros encontrados:
{'svc__C': 2, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}

Métricas en entrenamiento (CV):
Mejor Balanced Accuracy: 0.5888

Reporte de clasificación en Test:
              precision    recall  f1-score   support

           0       0.68      0.75      0.71        48
           1       0.71      0.64      0.67        47

    accuracy                           0.69        95
   macro avg       0.70      0.69      0.69        95
weighted avg       0.70      0.69      0.69        95



1. Rendimiento moderado del modelo
Exactitud en prueba (69%): El modelo tiene un rendimiento aceptable pero no sobresaliente con datos no vistos.

Precisión equilibrada (58,88% en entrenamiento): Sugiere que el modelo tuvo dificultades con el equilibrio de clases durante el entrenamiento, pero mejora en los datos de prueba.

2. Desempeño por clase
Clase 0 (ej. clase negativa):

Recall/Sensibilidad alto (75%): El modelo identifica mejor los verdaderos negativos (ej. pocos falsos negativos).

Precisión más baja (68%): Genera algunos falsos positivos (incorrectamente etiqueta ejemplos como Clase 0).

Clase 1 (ej. clase positiva):

Precisión más alta (71%): Menos falsos positivos para esta clase.

Recall/Sensibilidad más bajo (64%): El modelo no detecta todos los verdaderos positivos.

3. Generalización
El modelo funciona mejor en prueba (69% de exactitud) que durante la validación cruzada (58,88% de precisión equilibrada), lo que podría indicar:

Regularización exitosa (evita sobreajuste).

Variabilidad en los datos de entrenamiento (ej. pliegues de CV con muestras más difíciles).

4. Balance de clases
El conjunto de datos está equilibrado (48 vs. 47 ejemplos para Clase 0 y 1).

F1 macro (69%): Confirma que ninguna clase domina el rendimiento.

5. Áreas de mejora
Mejorar el recall de la Clase 1: Analizar por qué se pierden verdaderos positivos (ej. ingeniería de características, ajuste de umbrales).

Ajuste de hiperparámetros: Priorizar métricas equilibradas (ej. usar class_weight en modelos como SVM o árboles).

Modelos avanzados: Probar algoritmos como XGBoost o redes neuronales.

Conclusión final
El modelo es funcional pero no óptimo. Enfócate en mejorar la detección de la Clase 1 (recall) y refinar el entrenamiento para aumentar su robustez.

## Cluster 5

In [35]:

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import make_classification

cluster=5

# Generar datos sintéticos (cambia esto por tu dataset real)
df_cluster = df_final[df_final['Cluster'] == cluster].copy()

# Calcular el tercer cuartil (Q3)
q3 = df_cluster['Stream'].quantile(0.75)

# Definir las categorías 0 para las canciones con menos de q3 de stream y 1 para las de mayor de q3, estás serían las más exitosas
bins = [0, q3, df_cluster['Stream'].max()]
labels = [0, 1]

df_cluster['Categoria'] = pd.cut(df_cluster['Stream'], bins=bins, labels=labels, include_lowest=True)

# Ver la distribución de las categorías
print(df_cluster['Categoria'].value_counts())



# Verificar si hay suficientes muestras en la categoría 1
count_cat1 = df_cluster['Categoria'].value_counts().get(1, 0)



# Filtrar por categorías
categoria_0 = df_cluster[df_cluster['Categoria'] == 0]
categoria_1 = df_cluster[df_cluster['Categoria'] == 1]

# Muestrear la categoría 0 con el mismo número de elementos que la categoría 1
categoria_0_sample = categoria_0.sample(n=min(count_cat1, len(categoria_0)), random_state=42)

# Concatenar para obtener un dataset balanceado
df_balanceado = pd.concat([categoria_0_sample, categoria_1]).sample(frac=1, random_state=42).reset_index(drop=True)
print(df_balanceado['Categoria'].value_counts())

# Definir variables predictoras y objetivo
X = df_balanceado[['Acousticness', 'Danceability', 'Duration_min', 'Energy',
                       'Instrumentalness', 'Key', 'Liveness', 'Loudness',
                       'Speechiness', 'Tempo', 'Valence']]
y = df_balanceado['Categoria']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Categoria
0    593
1    198
Name: count, dtype: int64
Categoria
0    198
1    198
Name: count, dtype: int64


In [36]:
# Importar librerías
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# 1. Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 2. Crear pipeline con LDA
pipeline = Pipeline([
    ('lda', LinearDiscriminantAnalysis())
])

# 3. Definir parámetros para Grid Search
param_grid = {
    'lda__solver': ['svd', 'lsqr', 'eigen'],  # Métodos de solución
    'lda__shrinkage': [None, 'auto', 0.1, 0.5, 0.9]  # Solo para 'lsqr' y 'eigen'
}

# 4. Configurar Grid Search con validación cruzada estratificada
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='balanced_accuracy',  # Métrica clave para optimizar
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),  # 5 folds estratificados
    n_jobs=-1,  # Usar todos los núcleos del procesador
    verbose=1  # Mostrar progreso
)

# 5. Ejecutar Grid Search en datos de entrenamiento
grid_search.fit(X_train, y_train)

# 6. Evaluar resultados
print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)

print("\nMétricas en entrenamiento (CV):")
print(f"Mejor Balanced Accuracy: {grid_search.best_score_:.4f}")

# 7. Evaluar en conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nReporte de clasificación en Test:")
print(classification_report(y_test, y_pred))

# 8. (Opcional) Guardar modelo entrenado
# import joblib
# joblib.dump(best_model, 'mejor_modelo_lda.pkl')


Fitting 5 folds for each of 15 candidates, totalling 75 fits

Mejores parámetros encontrados:
{'lda__shrinkage': None, 'lda__solver': 'svd'}

Métricas en entrenamiento (CV):
Mejor Balanced Accuracy: 0.5824

Reporte de clasificación en Test:
              precision    recall  f1-score   support

           0       0.59      0.47      0.53        40
           1       0.56      0.68      0.61        40

    accuracy                           0.57        80
   macro avg       0.58      0.57      0.57        80
weighted avg       0.58      0.57      0.57        80



 Conclusiones del Modelo Linear Discriminant Analysis (LDA)

1️⃣ Precisión global moderada (57%)

La accuracy del modelo en el conjunto de prueba es 0.57, lo que indica que el modelo clasifica correctamente el 57% de los casos.

No es un resultado óptimo, pero puede ser aceptable dependiendo del problema.

2️⃣ El modelo favorece ligeramente la clase 1

Clase 0: Precisión 0.59, Recall 0.47, F1-score 0.53.
Clase 1: Precisión 0.56, Recall 0.68, F1-score 0.61.
La clase 1 tiene mejor recall (0.68), lo que indica que el modelo detecta bien los casos positivos, pero con menor precisión.
La clase 0 tiene recall bajo (0.47), lo que sugiere que hay muchos falsos negativos en esta clase.

## Conclusión final

Los modelos muestran una ligera inclinación a identificar la categoría 1, aunque sin una tendencia contundente. Este aspecto es relevante, ya que este tipo de estudio podría guiar a un artista en la creación de una canción exitosa. La estrategia que mejor favoreció el análisis de los datos fue la agrupación en clústeres por género, donde se observó un mejor desempeño del F1-score y el recall. Se recomienda utilizar bases de datos más grandes y con menor dispersión temporal. Además, realizar análisis más específicos en géneros similares permitiría construir clústeres más sólidos y representativos.